In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [ ]:
%pylab inline

### Пусть вам дан следующий SQL-запрос
```sql

select * from logs l join ips r
on l.ip == r.ip
```

### Таблица `logs` находится в HDFS по пути `/user/pavel.klemenkov/lectures/lecture02/data/logsM.txt`, таблица `ips`  находится в HDFS по пути `/user/pavel.klemenkov/lectures/lecture02/data/ipDataM.txt`

## 1. Выполните этот запрос в Spark SQL, предварительно зарегистрировав таблицы

In [13]:
sql = """
select url, code, region from logs as l join ips as r
on l.ip == r.ip
"""

In [2]:
from pyspark.sql.types import *

In [3]:
log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", LongType()),
    StructField("url", StringType()),
    StructField("size", IntegerType()),
    StructField("code", IntegerType()),
    StructField("ua", StringType())
])

In [4]:
log = spark.read.csv("/user/pavel.klemenkov/lectures/lecture02/data/logsM.txt", sep="\t", schema=log_schema)

In [5]:
log.show(5)

+--------------+--------------+--------------------+----+----+--------------------+
|            ip|     timestamp|                 url|size|code|                  ua|
+--------------+--------------+--------------------+----+----+--------------------+
| 33.49.147.163|20140101014611|http://news.rambl...| 378| 431|Safari/5.0 (compa...|
|197.72.248.141|20140101020306|http://news.mail....|1412| 203|Safari/5.0 (compa...|
| 33.49.147.163|20140101023103|http://lenta.ru/4...|1189| 451|Chrome/5.0 (compa...|
| 75.208.40.166|20140101032909|http://newsru.com...|  60| 306|Safari/5.0 (Windo...|
|197.72.248.141|20140101033626|http://newsru.com...| 736| 307|Chrome/5.0 (compa...|
+--------------+--------------+--------------------+----+----+--------------------+
only showing top 5 rows



In [6]:
ip_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("region", StringType())
])

In [7]:
ips = spark.read.csv("/user/pavel.klemenkov/lectures/lecture02/data/ipDataM.txt", sep="\t",
                     schema=ip_schema)

In [8]:
ips.show(5)

+--------------+------------------+
|            ip|            region|
+--------------+------------------+
|  49.105.15.79|              Komi|
|110.91.102.196|Chelyabinsk Oblast|
|56.167.169.126|  Saint Petersburg|
| 75.208.40.166|  Ulyanovsk Oblast|
|168.255.93.197|    Irkutsk Oblast|
+--------------+------------------+
only showing top 5 rows



In [9]:
log.registerTempTable("logs")

In [10]:
ips.registerTempTable("ips")

In [14]:
sql

'\nselect url, code, region from logs as l join ips as r\non l.ip == r.ip\n'

In [15]:
spark.sql(sql).show(5)

+--------------------+----+--------------------+
|                 url|code|              region|
+--------------------+----+--------------------+
|http://news.rambl...| 431|      Irkutsk Oblast|
|http://news.rambl...| 431|         Omsk Oblast|
|http://news.rambl...| 431|Nizhny Novgorod O...|
|http://news.rambl...| 431|              Jewish|
|http://news.rambl...| 431|      Kamchatka Krai|
+--------------------+----+--------------------+
only showing top 5 rows



## 2. Теперь перепешите этот запрос с использованием DataFrame API

In [16]:
log.join(ips, on="ip", how="inner").select("url", "code", "region").show(5)

+--------------------+----+--------------------+
|                 url|code|              region|
+--------------------+----+--------------------+
|http://news.rambl...| 431|      Irkutsk Oblast|
|http://news.rambl...| 431|         Omsk Oblast|
|http://news.rambl...| 431|Nizhny Novgorod O...|
|http://news.rambl...| 431|              Jewish|
|http://news.rambl...| 431|      Kamchatka Krai|
+--------------------+----+--------------------+
only showing top 5 rows



## 3. Посчитайте гистограмму распределения кодов ответа в логе доступа. Отсуртируйте результат по каунту. Наибольшая сумма сверху

In [17]:
import pyspark.sql.functions as f

In [18]:
log.groupBy("code").agg(f.count("code").alias("code_count"))\
   .orderBy("code_count", ascending=False).show(5)

+----+----------+
|code|code_count|
+----+----------+
| 302|       323|
| 307|       182|
| 205|       181|
| 502|       181|
| 504|       177|
+----+----------+
only showing top 5 rows



## 4. Изобразите гистограмму кодов на графике

In [ ]:
# Ваш код здесь

## 5. Напишите запрос, который вернет распределение доменов среди запросов

In [ ]:
# Ваш код здесь